<a href="https://colab.research.google.com/github/aobd1356/Homework/blob/main/EX04_03_%E5%AE%A2%E6%88%B6%E5%88%86%E7%BE%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 問題定義

將 性別 欄位 Label Encoding

將客戶分成 3 群，並解讀各群特質

計算 k = 2 ~ 15 的 Calinski-Harbasz Score，找出最佳 k 值

## 資料收集

In [11]:
!wget -O car_models.csv https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv

--2025-05-27 07:47:25--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2839 (2.8K) [text/plain]
Saving to: ‘car_models.csv’

car_models.csv      100%[===================>]   2.77K  --.-KB/s    in 0s      

2025-05-27 07:47:25 (32.7 MB/s) - ‘car_models.csv’ saved [2839/2839]



In [12]:
import pandas as pd
url='https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv'
df=pd.read_csv(url)
df.head()

,性別,年齡,收入（千）,消費指數（1~100）
0,女,74,38,81
1,女,51,71,91
2,女,30,65,10
3,女,88,49,17
4,女,55,48,70


## 資料前處理

### 資料清理

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   性別           200 non-null    object
 1   年齡           200 non-null    int64 
 2   收入（千）        200 non-null    int64 
 3   消費指數（1~100）  200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 6.4+ KB


### 探索性分析

In [14]:
df_cor=df.drop(columns='性別').corr()
df_cor

,年齡,收入（千）,消費指數（1~100）
年齡,1.000000,0.031519,-0.127454
收入（千）,0.031519,1.000000,0.031476
消費指數（1~100）,-0.127454,0.031476,1.000000


### 資料分割

In [15]:
df['性別'].unique()

array(['女', '男'], dtype=object)

### 類別轉換

In [16]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['女', '男']])
df[['性別']] = encoder.fit_transform(df[['性別']])
df

,性別,年齡,收入（千）,消費指數（1~100）
0,0.0,74,38,81
1,0.0,51,71,91
2,0.0,30,65,10
3,0.0,88,49,17
4,0.0,55,48,70
...,...,...,...,...
195,1.0,86,84,82
196,1.0,59,52,30
197,0.0,63,29,61
198,1.0,67,80,9


### 特徵縮放

## 模型訓練

In [17]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(df)

KMeans(n_clusters=3)

In [18]:
df['cluster'] = kmeans.labels_
df

,性別,年齡,收入（千）,消費指數（1~100）,cluster
0,0.0,74,38,81,1
1,0.0,51,71,91,1
2,0.0,30,65,10,2
3,0.0,88,49,17,0
4,0.0,55,48,70,1
...,...,...,...,...,...
195,1.0,86,84,82,1
196,1.0,59,52,30,0
197,0.0,63,29,61,0
198,1.0,67,80,9,2


## 模型評估

In [19]:
df.groupby('cluster').mean()

,性別,年齡,收入（千）,消費指數（1~100）
cluster,,,,
0,0.574074,56.314815,38.074074,29.203704
1,0.570000,48.940000,60.280000,77.820000
2,0.565217,57.108696,80.021739,25.065217


In [20]:
from sklearn.metrics import calinski_harabasz_score
score = calinski_harabasz_score(df.drop(columns='cluster'), kmeans.labels_)
score

np.float64(83.36523053972887)

## 模型調整

In [21]:
df_nocluster=df.drop(columns='cluster')
for i in range(2, 16):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(df_nocluster)
    score = calinski_harabasz_score(df_nocluster, kmeans.labels_)
    print(f'k={i}, score={score}')

k=2, score=111.68583137120146
k=3, score=97.18095240094486
k=4, score=78.44070103882649
k=5, score=90.96785502651971
k=6, score=94.38297199289819
k=7, score=89.38120983994182
k=8, score=92.52977530053992
k=9, score=97.87142038200732
k=10, score=93.80322199561341
k=11, score=91.86437972757437
k=12, score=87.58356018516174
k=13, score=85.14563662732405
k=14, score=84.69166006069216
k=15, score=85.72006350363601


## 模型部署

### 儲存模型

### 推論預測